<a href="https://colab.research.google.com/github/plumwiserim/Data-Analysis/blob/main/Class9_ML_ModelSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day4. 파이썬 머신러닝 실습
## 교차 검증 - KFold

In [ ]:
# 모델 구현을 위한 라이브러리 로딩
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# load_iris() 메서드를 이용하여 iris 데이터 셋 로드
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB


In [ ]:
iris_df['label']  = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [ ]:
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=3)
n_iter = 0

for train_index, test_index in kfold.split(iris_df):
    n_iter+=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n##교차 검증: {0}'.format(n_iter))
    print('학습 레이블 분포: \n', label_train.value_counts())
    print('검증 레이블 분포: \n', label_test.value_counts())


##교차 검증: 1
학습 레이블 분포: 
 2    50
1    50
Name: label, dtype: int64
검증 레이블 분포: 
 0    50
Name: label, dtype: int64

##교차 검증: 2
학습 레이블 분포: 
 2    50
0    50
Name: label, dtype: int64
검증 레이블 분포: 
 1    50
Name: label, dtype: int64

##교차 검증: 3
학습 레이블 분포: 
 1    50
0    50
Name: label, dtype: int64
검증 레이블 분포: 
 2    50
Name: label, dtype: int64


## 교차 검증 - StratifiedKFold

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skf.split(iris_df[iris_df.columns], iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n## 교차검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포: \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())


## 교차검증: 1
학습 레이블 데이터 분포: 
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    17
0    17
2    16
Name: label, dtype: int64

## 교차검증: 2
학습 레이블 데이터 분포: 
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 2    17
0    17
1    16
Name: label, dtype: int64

## 교차검증: 3
학습 레이블 데이터 분포: 
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 2    17
1    17
0    16
Name: label, dtype: int64


In [ ]:
# load_iris() 메서드를 이용하여 iris 데이터 셋 로드
iris = load_iris()
#data = iris.data
label = iris.target

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = label

# 붓꽃 데이터 세트에서 Stratified Kfold 를 이용한 검증
dt_clf = DecisionTreeClassifier()

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

In [ ]:
iris_df.shape

(150, 5)

In [ ]:
# StratifiedKFold의 split 호출 시 반드시 레이블 데이터 세트도 추가 입력 필요
for train_index, test_index in skfold.split(iris_df[iris_df.columns], iris_df['label']):

    #split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = iris_df.iloc[train_index], iris_df.iloc[test_index]
    y_train, y_test = label[train_index], label[test_index]

    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)

    # 반복 시마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred),4)
    
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('{0} 교차검증 정확도: {1}, 학습데이터 크기: {2}, 검증데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)
    
# 교차검증별 정확도 및 평균정확도 계산
print('\n교차검증별 정확도: ', cv_accuracy)
print('평균 검증 정확도: ', np.mean(cv_accuracy))

1 교차검증 정확도: 1.0, 학습데이터 크기: 100, 검증데이터 크기: 50
2 교차검증 정확도: 1.0, 학습데이터 크기: 100, 검증데이터 크기: 50
3 교차검증 정확도: 1.0, 학습데이터 크기: 100, 검증데이터 크기: 50

교차검증별 정확도:  [1.0, 1.0, 1.0]
평균 검증 정확도:  1.0


### cross_val_score

In [ ]:
# K Fold Validation을 위한 cross_val_score() 메서드 로딩
from sklearn.model_selection import cross_val_score

# 모델 구현을 위한 라이브러리 로딩
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
import numpy as np

In [ ]:
# load_iris() 메서드를 이용하여 iris 데이터 셋 로드
iris = load_iris()
data = iris.data
label = iris.target

In [ ]:
# DecisionTreeClassifier 모델 객체 생성
dt_clf = DecisionTreeClassifier(random_state=156)

In [ ]:
# cross_val_score() 메서드를 이용하여 교차 검증 수행
scores = cross_val_score(estimator=dt_clf,
                         X=data, 
                         y=label,
                         scoring='accuracy',
                         cv=3)

In [ ]:
# 교차 검증 수행 결과 성능 지표 출력
print('Fold val accuracy:',np.round(scores, 4))
print('Avg val accuracy:', np.round(np.mean(scores), 4))

Fold val accuracy: [0.98 0.94 0.98]
Avg val accuracy: 0.9667


### GridSearchCV
- estimator: 구현하고자 하는 모델(Classification, Regression)
- param_grid: 모델의 튜닝에 사용될 파라미터 정보, 딕셔너리
- scoring: 검증 지표(성능 평가 지표)
- cv: Cross Validation의 Fold 숫자
- refit: 최적의 파라미터로 모델을 재 학습 시킬지 여부


In [ ]:
# 하이퍼파라미터 튜닝을 을 위한 GridSearchCV 라이브러리 로딩
from sklearn.model_selection import GridSearchCV

# 모델 구현을 위한 라이브러리 로딩
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# load_iris() 메서드를 이용하여 iris 데이터 셋 로드
iris = load_iris()

# 학습, 테스트 데이터셋 분리
x_train, x_test, y_train, y_test = train_test_split(iris.data, 
                                                    iris.target, 
                                                    test_size=0.2,
                                                    random_state=121)

In [ ]:
# DecisionTreeClassifier 모델 객체 생성
dtree = DecisionTreeClassifier()

# 모델의 후보 파라미터 셋(param_grid)을 지정한 딕셔너리 객체 생성
parameters = {'max_depth':[1,2,3],
              'min_samples_split':[2,3]}

In [ ]:
# GridSearchCV 객체 생성
grid_dtree = GridSearchCV(estimator=dtree,
                          param_grid=parameters,
                          cv=3,
                          refit=True)

In [ ]:
# GridSearchCV 객체의 fit() 메서드를 이용하여
# 후보 파라미터 셋의 성능 검증
grid_dtree.fit(x_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]},

In [ ]:
# 후보 파라미터 셋의 성능 검증 결과 출력
print('Optimal parameter:', grid_dtree.best_params_)
print('Max accuracy: {0:.4f}'.format(grid_dtree.best_score_))

Optimal parameter: {'max_depth': 3, 'min_samples_split': 2}
Max accuracy: 0.9750


In [ ]:
# 최적의 파라미터 모델을 이용하여 예측값 생성
estimator = grid_dtree.best_estimator_
pred = estimator.predict(x_test)


# 최적의 파라미터 모델의 성능지표 출력
print('Test accuracy: {0:.4f}'.format(accuracy_score(y_test,pred))) 


Test accuracy: 0.9667


### 타이타닉 데이터셋

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

url = 'https://codepresso-online-platform-public.s3.ap-northeast-2.amazonaws.com/learning-resourse/titanic_train.csv'
titanic_df = pd.read_csv(url)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 데이터 전처리 기능 모듈화

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1, inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [ ]:
# 원본 데이터 후 transform_features 함수 호출하여 전처리 

titanic_df = pd.read_csv(url)
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived',axis=1)

X_titanic_df = transform_features(X_titanic_df)
X_titanic_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.0,1,0,7.2500,7,3
1,1,0,38.0,1,0,71.2833,2,0
2,3,0,26.0,0,0,7.9250,7,3
3,1,0,35.0,1,0,53.1000,2,3
4,3,1,35.0,0,0,8.0500,7,3


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X_titanic_df,
                                                  y_titanic_df,
                                                  test_size=0.2,
                                                 random_state=10)

In [ ]:
# 필요한 라이브러리 로딩
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#모델 객체 생성 및 학습과 예측 수행
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
lr_clf = LogisticRegression(max_iter=300)

# 각 모델 별 학습 및 예측 수행
dt_clf.fit(X_train , y_train)
dt_pred = dt_clf.predict(X_test)

rf_clf.fit(X_train , y_train)
rf_pred = rf_clf.predict(X_test)

lr_clf.fit(X_train , y_train)
lr_pred = lr_clf.predict(X_test)

In [ ]:
# 필요한 라이브러리 로딩
from sklearn.metrics import accuracy_score

# 성능 결과 출력
print('DecisionTreeClassifier : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('RandomForestClassifier :{0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('LogisticRegression : {0:.4f}'.format(accuracy_score(y_test, lr_pred)))


DecisionTreeClassifier : 0.7877
RandomForestClassifier :0.8324
LogisticRegression : 0.8268


### Q. KFold 적용하기

In [ ]:
from sklearn.model_selection import KFold

def exec_kfold(clf, folds=5):
    kfold = KFold(n_splits=folds)
    scores = []
    
    for iter_count , (train_index, test_index) in enumerate(kfold.split(X_titanic_df)):
        X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
        y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]
        
        clf.fit(X_train, y_train) 
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print("교차 검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))     
    
    mean_score = np.mean(scores)
    print("평균 정확도: {0:.4f}".format(mean_score)) 

exec_kfold(dt_clf , folds=5) 

교차 검증 0 정확도: 0.7374
교차 검증 1 정확도: 0.7753
교차 검증 2 정확도: 0.7865
교차 검증 3 정확도: 0.7640
교차 검증 4 정확도: 0.8090
평균 정확도: 0.7745


### Q. cross_val_score 적용하기

In [ ]:
from sklearn.model_selection import cross_val_score

# cross_val_score 사용하여 교차 검증 수행
# type 12
scores = cross_val_score(dt_clf, X_titanic_df , y_titanic_df , cv=5)

print(scores)
print("평균 정확도: {0:.4f}".format(np.mean(scores)))

[0.73743017 0.78651685 0.79775281 0.7752809  0.8258427 ]
평균 정확도: 0.7846


### Q. GridSearchCV 적용하기

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[2,3,5,10],
             'min_samples_split':[2,3,5], 
              'min_samples_leaf':[1,5,8]}

# GridSearchCV 활용한 모델 학습
# type 13
grid_dclf = GridSearchCV(dt_clf , param_grid=parameters , scoring='accuracy' , cv=5)
grid_dclf.fit(X_train , y_train)

print('best_params_ : ',grid_dclf.best_params_)
print('best_score_: ', grid_dclf.best_score_)

best_dclf = grid_dclf.best_estimator_
pred = best_dclf.predict(X_test)
print('accuracy: ', accuracy_score(y_test , pred))


best_params_ :  {'max_depth': 5, 'min_samples_leaf': 8, 'min_samples_split': 5}
best_score_:  0.813158672313602
accuracy:  0.8212290502793296
